In [1]:
from collections import defaultdict
from PIL import Image
from pathlib import Path
import imagehash
from tqdm import tqdm
import shutil

In [2]:
def find_similar_images(
    image_paths: list[Path],
    threshold: int = 5
) -> dict[str, list[Path]]:
    """
    Группирует похожие изображения на основе хеша.
    
    :param image_paths: список путей к изображениям
    :param threshold: максимальное расстояние между хешами (0-64)
                     чем меньше, тем строже сравнение
    :return: {representative_hash: [список похожих путей]}
    """
    hashes = {}
    groups = defaultdict(list)
    
    # Сначала собираем все хеши
    for path in tqdm(image_paths, desc="Compute hashes"):
        try:
            img = Image.open(path)
            h = imagehash.phash(img)
            hashes[path] = h
        except Exception as e:
            print(f"Ошибка обработки {path}: {e}")
    
    # Группируем похожие изображения
    used_paths = set()
    for path1, hash1 in tqdm(hashes.items(), desc="Grouping similar images"):
        if path1 in used_paths:
            continue
            
        current_group = [path1]
        for path2, hash2 in hashes.items():
            if path2 not in used_paths and path1 != path2:
                if hash1 - hash2 <= threshold:  # Расстояние Хемминга
                    current_group.append(path2)
        
        if len(current_group) > 1:  # Только группы с похожими изображениями
            for p in current_group:
                used_paths.add(p)
            # Используем hex-представление хэша как ключ
            groups[str(hash1)].extend(current_group)
    
    return dict(groups)

In [3]:
def find_and_move_duplicates(
    input_dir: Path,
    output_dir: Path,
    threshold: int = 5,
    min_group_size: int = 2
) -> int:
    """
    Находит дубликаты в папке и перемещает их в подпапки output_dir.
    
    :param input_dir: Папка с изображениями для проверки
    :param output_dir: Папка для сохранения групп дубликатов
    :param threshold: Максимальное расстояние между хешами (0-64)
    :param min_group_size: Минимальный размер группы для перемещения
    """
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    # Находим похожие изображения
    groups = find_similar_images(sorted(input_dir.iterdir()), threshold)
    
    # Перемещаем группы дубликатов
    moved = 0
    for group_num, (hash_val, paths) in enumerate(groups.items(), 1):
        if len(paths) >= min_group_size:
            # Первый файл в группе считаем оригиналом (не перемещаем)
            original = paths[0]
            filename_without_suffix = original.name.replace(original.suffix, '')
            group_dir = output_dir / f"duplicates_{filename_without_suffix}_{group_num}"
            group_dir.mkdir(exist_ok=True)
            
            # Перемещаем дубликаты
            for duplicate in paths[1:]:
                shutil.move(duplicate, group_dir / duplicate.name)
                # print(f"Дубликат → {group_dir.name}/{duplicate.name}")
                moved += 1
    
    return moved

In [6]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir.parent.parent / "data"
print(f"data_dir: {data_dir}")

dataset_dir = data_dir / "interior_dataset"
for path in sorted(dataset_dir.iterdir()):
    if path.is_dir():
        moved = find_and_move_duplicates(input_dir=path, output_dir=path / "dublicates")
        print(f"{path.name} | Moved {moved}")

Директория: /home/little-garden/CodeProjects/InteriorClass/src/notebooks
data_dir: /home/little-garden/CodeProjects/InteriorClass/data


 69%|██████▉   | 10475/15172 [00:19<00:07, 591.95it/s]/home/little-garden/CodeProjects/InteriorClass/venv3.12/lib/python3.12/site-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 15172/15172 [00:27<00:00, 550.17it/s]


Ошибка обработки /home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/A0/dublicates: [Errno 21] Is a directory: '/home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/A0/dublicates'


100%|██████████| 15171/15171 [05:11<00:00, 48.70it/s]



Оригинал: A0_01639_a5aa.jpg

Оригинал: A0_04096_3e70.jpg
A0 | Moved 2


...


Оригинал: D0_01096_7228.jpg

Оригинал: D0_01491_7c4f.jpg

Оригинал: D0_03054_05ae.jpg

Оригинал: D0_03253_a317.jpg

Оригинал: D0_03672_bc0d.jpg

Оригинал: D0_04052_8520.jpg

Оригинал: D0_04356_942c.jpg

Оригинал: D0_04780_7931.jpg

Оригинал: D0_05267_ea24.jpg

Оригинал: D0_05428_3e97.jpg

Оригинал: D0_05694_c042.jpg

Оригинал: D0_07771_3b26.jpg
D0 | Moved 12


100%|██████████| 4226/4226 [00:08<00:00, 528.17it/s]


Ошибка обработки /home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/D1/dublicates: [Errno 21] Is a directory: '/home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/D1/dublicates'


100%|██████████| 4225/4225 [00:23<00:00, 182.65it/s]


Оригинал: D1_00141_b1d1.jpg

Оригинал: D1_00330_1839.jpg

Оригинал: D1_00743_e644.jpg

Оригинал: D1_01008_8c4d.jpg

Оригинал: D1_02384_7778.jpg

Оригинал: D1_02676_1a48.jpg

Оригинал: D1_03184_d2d3.jpg
D1 | Moved 7
